# Forest Inventory and Analysis `10 points`

Source:

* Dataset: https://apps.fs.usda.gov/fia/datamart/datamart.html
* Documentation: https://www.fia.fs.fed.us/library/database-documentation/index.php
        
Description from [Data Is Plural](https://www.data-is-plural.com/archive/2019-08-21-edition/):

> The U.S. Forest Service’s Forest Inventory and Analysis program tracks “trends in forest area and location; in the species, size, and health of trees; in total tree growth, mortality, and removals by harvest; in wood production and utilization rates by various products; and in forest land ownership.” It also “serves as perhaps the largest publicly available” dataset of “downed and dead wood.” The inventory is available to download and comes with user guides.

**Topics:**

* Downloading files
* Opening Excel files
* Using parameters when opening Excel files
* When to do things manually vs doing things with code

## Automatic downloading `2 points`

If you want to download files for Excel, you need to go to [this page](https://apps.fs.usda.gov/fia/datamart/datamart_excel.html) and click on the map. It leads you to a file like `https://apps.fs.usda.gov/fia/datamart/Workbooks/IL.xlsm`. Awful user interface!

Instead, I want you to use `requests` and a `for` loop to download all of the states automatically. You might find [this SO answer](https://stackoverflow.com/questions/44699682/how-to-save-a-file-to-a-specific-directory-in-python) useful.

*Note that the page says they don't have information for every state.*

In [78]:
## import the nessary libaris
import requests
from bs4 import BeautifulSoup
import pandas as pd


pd.set_option("display.max_colwidth",500)

In [79]:
# To save to a relative path.
response = requests.get('https://apps.fs.usda.gov/fia/datamart/datamart_excel.html')  
soup = BeautifulSoup(response.text, 'html.parser')

# Since there are several links for each state we need to get the different Links to download each file from
pages = soup.findAll('area')

# Making a list of dictionaries
rows=[]
for p in pages:
    row={}
    row['Link']=p.get('href')
    row['State']=p.get('title')
    rows.append(row)
rows

# turn it into Dataframe
States = pd.DataFrame(rows)

# add "https://apps.fs.usda.gov/fia/datamart" in front of the first Link column to get the complete url
# use + to add a string in front of the whole columns
States.Link = 'https://apps.fs.usda.gov/fia/datamart/'+States.Link

In [83]:
States.Link[0]

'https://apps.fs.usda.gov/fia/datamart/Workbooks\\AS.xlsm'

[Documentation of **open** function](https://docs.python.org/3/library/functions.html#open)

In [113]:
# for each row in the dataframe, get me the url in the first column and download the file with the second column as the name

#for state in States:
url= States.Link[0]
# Get the link or url
r = requests.get(url,allow_redirects=True)  
# Save the content with name
open('Forestry Downloads/test.xlsm', 'wb').write(r.content)


##### file does not open!! weird
## how to download if link is not working???

531

## Reading in the data `3 points`

### Read in the data for Virginia

**We're interested in sheet `SR004`**, which explains how many acres cover each type of ownership.

Read the file in so that it the dataset looks like this:

|Forest type group|Total|National Forest|Other federal|State and local|Private|
|---|---|---|---|---|---|
|Total|16025876|1688425.0|518217.0|657963.0|13161271|
|...|...|...|...|...|...|
|Nonstocked|81574|0.0|1590.0|0.0|79984|

and your index goes up to `15`.

In [141]:
# import the file from Virgina, go to the sheet name, have NA values, define the colors, how many rows to skip
virgina_df = pd.read_excel('Forestry Downloads/VA.xlsm', sheet_name='SR004',na_values=['NA'],usecols="A:F",skiprows=range(11),nrows=16)

In [144]:
virgina_df.style.hide_index()

Forest type group,Total,National Forest,Other federal,State and local,Private
Total,16025876,1688425,518217,657963,13161271
White / red / jack pine group,171292,33764,2534,-,134995
Spruce / fir group,7735,-,-,6188,1547
Longleaf / slash pine group,10293,-,-,-,10293
Loblolly / shortleaf pine group,3038306,63540,79536,89038,2806193
Other eastern softwoods group,75076,-,-,5876,69201
Exotic softwoods group,4157,-,-,-,4157
Oak / pine group,1649711,140950,58413,53515,1396832
Oak / hickory group,9755134,1375367,314345,405917,7659506
Oak / gum / cypress group,373717,2939,40461,21746,308570


### Read in the data for South Dakota

You'll have fewer rows in this dataset than for Virginia.

In [142]:
# import the file from Virgina, go to the sheet name, have NA values, define the colors, how many rows to skip
sd_df = pd.read_excel('Forestry Downloads/SD.xlsm', sheet_name='SR004',na_values=['NA'],usecols="A:F",skiprows=range(11),nrows=14)

In [143]:
sd_df.style.hide_index()

Forest type group,Total,National Forest,Other federal,State and local,Private
Total,1897358,993588,60164,96499,747106
White / red / jack pine group,6098,-,-,-,6098
Spruce / fir group,85957,60844,17348,-,7765
Other eastern softwoods group,52741,-,6331,-,46411
Pinyon / juniper group,71273,17366,5920,-,47987
Ponderosa pine group,1023191,699839,4748,63442,255163
Oak / pine group,9775,-,-,4340,5435
Oak / hickory group,156796,24422,5978,7412,118985
Elm / ash / cottonwood group,140167,-,-,12700,127468
Maple / beech / birch group,8449,-,-,-,8449


# Calculations `1 point`

## What percent of forested land is a "National Forest" in South Dakota vs Virginia?

You can do this calculation manually. Pay special attention to column names.

In [158]:
# select with iloc[row number, column number]

print('In Virgina',(virgina_df.iloc[0, 2]/virgina_df.iloc[0, 1]).round(2)*100,'% of the forested land is a National Forest and in South Dakota it is',(sd_df.iloc[0, 2]/sd_df.iloc[0, 1]).round(2)*100,'%')

In Virgina 11.0 % of the forested land is a National Forest and in South Dakota it is 52.0 %


## What percent of forested land is privately owned in SD vs VA?

In [159]:
# select with iloc[row number, column number]

print('In Virgina',(virgina_df.iloc[0, 5]/virgina_df.iloc[0, 1]).round(2)*100,'% of the forested land is privately owned and in South Dakota it is',(sd_df.iloc[0, 5]/sd_df.iloc[0, 1]).round(2)*100,'%')

In Virgina 82.0 % of the forested land is privately owned and in South Dakota it is 39.0 %


## Do the calculation for private ownership of all forests in South Dakota using only one line, and without typing the actual numbers `1 point`

Tip: `df.loc[0]` will be your friend

In [168]:
# You can use select the rows with the .

(sd_df.loc[0].Private/sd_df.loc[0].Total).round(2)

0.39

## Using the files you downloaded, calculate the private ownership rate for all forested land in each state `3 points`

> Tip: Use a for loop

In [ ]:
# need the names